# RandomForests for RNASeq expression classification

# 1. Introduction (Liting)

- Supervised classification
- Random forests (concept)
- Vocabulary (features, labels, classes)

# 2.1 Getting started (Liting)

- Introuduce libaries / Python / Pandas etc.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# 2.2 The dataset

In this session we will use an miRNA expression dataset that contains 714 expression signatures across 29 tumor/normal pairs measured in human cervical tissue. 

We will use this dataset to perform several pre-processing steps, feature scaling/normalization and to train a RandomForest classifier with Python.

First, let's import our dataset and have a look at the data.

In [3]:
try:
    exprm = pd.read_csv("./datasets/exprm.txt", sep="\t")
except Exception:
    print("An error occured while reading the data")

Print the first 5 rows.

In [4]:
exprm.head(n=5)

,N1,N2,N3,N4,N5,N6,N7,N8,N9,N10,...,T20,T21,T22,T23,T24,T25,T26,T27,T28,T29
let-7a,865,810,5505,6692,1456,588,9,4513,1962,10167,...,37,3174,116,1722,68,12121,14398,39196,198,1422
let-7a*,3,12,30,73,6,2,0,199,10,173,...,0,648,4,212,6,2,80,164,18,1
let-7b,975,2790,4912,24286,1759,508,33,6162,1455,18110,...,99,102358,184,28274,401,14471,24097,73139,669,2492
let-7b*,15,18,27,119,11,3,0,116,17,233,...,0,334,1,189,5,34,115,230,4,15
let-7c,828,1251,2973,6413,713,339,23,2002,476,3294,...,34,1711,22,3127,199,3186,1454,5883,167,321


### Question
- What is the dimension of the dataset above/ what type of numerical data does it contain?
- Which dimenson contains the features, which one the instances? 

#### TIp 
You can access a dataframe's dimension with the `dim()` function.

# 2.3 Pre-proccesing -  Normalization

A common normalization technique in RNASEq is the Median of Ratios method, which is used by [DESeq2](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-014-0550-8), one of the  major statistical tools for differential expression analysis. Briefly, this method corrects for differences in sequencing library size (number of reads), composition of the library (tissue specific expression) to make samples comparable. Without accounting for these confounding factors, a classifier might discriminate between samples based on technical variation which mask the true biological signal. 

Here we implement our own version of the Median of Ratios method. However, several more RNA sepecific normalization methods exist, such as Trimmed Mean (TMM) for example.


First, we perform a log transformation on the raw read counts to reduce the skewness of the dataset

In [18]:
exprm_log2 = exprm.apply(np.log2) # Perform a log transformation of our dataset.

In the following step, we calculate the mean for each miRNA expression across all samples

In [19]:
expr_means =  exprm_log2.apply(np.mean,1) # Calculate the geometric mean for each gene

Some samples might contains 0 (-inf) counts which we need to filter first before proceeding.

In [20]:
idxs = expr_means > 0 # Filter 0 Counts values
expr_means = expr_means[idxs] # Select geometric means with > 0 expression.

Next, we caclulate the row-wise mean to expression ratio for each sample across all miRNA. T

In [21]:
mean_ratio = exprm_log2[idxs].sub(expr_means,"index") # Calculate mean expr. ratios for each gene's expression per sample.

We select the media ration per miRNA which becomes our size factor. These size factors serve as a "pseudo" references to normalize our raw read_counts.

In [22]:
size_factors = mean_ratio.apply(lambda counts: np.median(counts),1) # Select median ratio for normalization

Finally, we divide each miRNA expression (per sample) by its corresponding size factor.

In [23]:
exprm_normalized = exprm[idxs].divide(np.exp(size_factors),'index')

,N1,N2,N3,N4,N5,N6,N7,N8,N9,N10,...,T20,T21,T22,T23,T24,T25,T26,T27,T28,T29
let-7a,461.133238,431.812627,2934.726560,3567.518645,776.196525,313.463981,4.797918,2405.889367,1045.946142,5420.048127,...,19.724774,1692.065777,61.839833,918.001660,36.250937,6461.729453,7675.602728,20895.466350,105.554198,758.071057
let-7b,824.339370,2358.878812,4152.979471,20533.236858,1487.192771,429.501949,27.900717,5209.824817,1230.167983,15311.575372,...,83.702151,86541.260737,155.567635,23904.996249,339.035987,12234.887201,20373.441841,61837.289406,565.623629,2106.926882
let-7c,519.774909,785.312090,1866.293240,4025.744551,447.583949,212.806394,14.438192,1256.750443,298.807798,2067.800180,...,21.343414,1074.075928,13.810444,1962.966351,124.921747,2000.003452,912.744827,3693.038390,104.833828,201.506939
let-7d,53.624715,74.017212,274.921074,1427.474808,141.992203,35.498051,0.755278,543.044649,154.076646,1076.270689,...,0.755278,2197.102761,4.531666,1782.455316,2.265833,399.541891,2151.786100,3728.805888,15.860831,116.312762
let-7e,113.030646,100.991879,527.030466,3879.827071,205.996680,80.927267,6.019383,1278.784582,136.439359,1968.338402,...,115.037107,6906.239327,208.671961,543.082155,25.415175,2842.486649,630.028806,1597.143087,46.148607,16.720510
let-7f,272.521920,91.958188,1674.884280,6938.053670,543.128045,171.463704,0.478949,2036.490695,549.833330,3838.296432,...,2.394744,12740.519513,27.779036,912.397642,16.763211,3435.021463,2942.183052,4973.884258,24.905342,148.474157
let-7g,228.290651,88.354100,981.598729,2074.533838,251.783649,63.328950,4.085739,533.699621,215.012000,1065.356373,...,4.596456,4377.358326,4.085739,374.866528,2.553587,718.579298,747.179469,1758.910521,6.639325,1062.802786
let-7i,192.462612,242.774416,728.323247,3088.186400,356.974223,71.075405,5.590200,510.305433,308.259620,1318.488685,...,10.381801,26880.079143,11.979001,2083.547535,105.415207,7548.367689,4727.712307,4531.256694,42.325803,120.588608
miR-100,93.418331,67.758360,275.042812,140.728903,21.650600,4.811245,16.438419,170.398244,6.815930,172.001992,...,0.400937,550.486562,1.202811,358.036780,2.806559,8.820615,638.692711,204.878830,2.004685,26.060908
miR-101,74.333529,126.226747,378.212734,377.277721,75.736048,35.530492,6.077584,166.432303,68.723451,179.054978,...,5.142571,997.658807,3.272545,294.061570,7.947610,162.692251,140.251941,139.784434,2.805039,5.610078


### Question
- Why did we perform a log transformation ?
- What is the rational behind size-factors ?



# 2.4 Pre-processing - Transformation

As a convention, most machine learning toolkits, such as Scikit-learn, expect the features of a dataset to be arranged in rows and attributes in columns.

This can be easily accomplished by Pandas's `transpose()` command.


In [97]:
exprm_normalized_t = exprm_normalized.transpose()

The expression dataset is currently missing a definition of which sample belongs to the tumor or normal class. 

For your convenience, we prepared a metadata file that contains the mampping between samples and biological condition.

In [98]:
try:
    exprm_meta = pd.read_csv("./datasets/exprm_meta.txt", sep="\t")
    exprm_meta = exprm_meta.set_index("sample")
except Exception:
    print("An error occured while reading the data")

To ensure that the labels (N,T) are assigned to the correct expression measurements, we merge both datasets by the index row which consists of the sample names, a shared column between both the expression and metadata.

In [99]:
exprm_all = pd.merge(exprm_normalized_t, exprm_meta, left_index=True, right_index=True)
exprm_all.head(n=5)

,let-7a,let-7b,let-7c,let-7d,let-7e,let-7f,let-7g,let-7i,miR-100,miR-101,...,miR-30d,miR-320a,miR-335,miR-423-5p,miR-424,miR-451,miR-484,miR-99a,miR-99b,condition
N1,461.133238,824.339370,519.774909,53.624715,113.030646,272.521920,228.290651,192.462612,93.418331,74.333529,...,102.579715,217.478852,76.826994,21.811653,7.749540,73.866171,14.870288,236.846561,17.781587,N
N2,431.812627,2358.878812,785.312090,74.017212,100.991879,91.958188,88.354100,242.774416,67.758360,126.226747,...,168.035343,950.165907,48.460104,62.918230,550.217320,100.598690,26.766519,250.616710,68.162751,N
N3,2934.726560,4152.979471,1866.293240,274.921074,527.030466,1674.884280,981.598729,728.323247,275.042812,378.212734,...,213.951977,853.865308,128.241982,68.790599,27.898343,1022.870601,56.507096,642.377445,71.867248,N
N4,3567.518645,20533.236858,4025.744551,1427.474808,3879.827071,6938.053670,2074.533838,3088.186400,140.728903,377.277721,...,503.129078,1393.148661,1154.768814,114.091724,533.684969,375.662243,44.610865,887.486095,114.098518,N
N5,776.196525,1487.192771,447.583949,141.992203,205.996680,543.128045,251.783649,356.974223,21.650600,75.736048,...,56.663081,678.919221,20.684191,99.830259,63.546226,73.162684,35.688692,114.292236,65.199153,N


### Question 
- What is the advantage of merging the two files by a common column? Couldn't we just simply 'copy & paste' the class labels? 
- What happend to the rows/columns of our dataframe after transposition? 

# 3. 1 Train and Test sets (Liting)
We need to split our dataset into a train and test-set. In a situation where no dedicated train/test sets are available, we usually reserve 2/3 of the initial dataset for training and 1/3 for testing
We need to define the size of our test set to split the dataset accordingly. The Random state ensures that our results are reproducible.


In [102]:
test_size = 0.33
random_state = 123

Split the dataset into train and test-set.

In [103]:
X_train, X_test, y_train, y_test = train_test_split(features, classes, test_size=test_size, random_state=random_state)


NameError: name 'features' is not defined

# 3.1 Random Forest  - Training (Tilman)

Create a new RandomForest instance with n trees.

In [ ]:
max_depth=2
random_state=123
n_trees=5000
rf = RandomForestClassifier(max_depth=2, 
                            random_state=random_state,
                            bootstrap=True,
                            max_features = 'sqrt',
                            n_estimators=n_trees)

# Criterion splitting/scoring
# Print one tree and explain decision path
# Plot one tree

We train our classifier on the testset that we previously created. In this steps, the actual "learning" takes place.

In [ ]:
rf.fit(X_train, y_train) #We should use OOB score here and plot it shomewhere.

# 3.2 Random Forest - Prediciting (Liting)

Let's predict the classes of our test-set

In [104]:
 Tilman

score = rf.predict(X_test) #Score predictions
probs = rf.predict_proba(X_test)[:, 1] #Score as class probs

NameError: name 'Tilman' is not defined

In [ ]:
print(sorted(zip(map(lambda x: round(x, 4), score), X_train.index),reverse=True))


# 3.3 Feature importances (Liting)

In [ ]:
print(sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), X_train.columns),reverse=True))

# 3.4 Measuring performance - Receiver operator curves (Tilman)

Calculate some performance stats.

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, probs)
roc_auc = roc_auc_score(y_test, probs)
# Only works for binary more 

In [ ]:
plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc) # https://stackoverflow.com/questions/34564830/roc-curve-with-sklearn-python
plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate or (1 - Specifity)')
plt.ylabel('True Positive Rate or (Sensitivity)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")

# 3.5 - Measuring performance - confusion matrices (Liting)

# Extra practice: 
- Linear classifier, logistic regression
- Compare results across classifiers
    

# More Ideas

In [ ]:
2. Add K-fold cross validation example
3. Plot OOB error across bags
4. Plot feature importances
5. Confusion matrix, feature selection
6. More classifiers
7. Imputation vs. missing data
8. Pruning
9. Spliting criterion
10. Feature selection
11. Advanced excercise / Out-of baggings error